In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('aapl.csv')
scaler = MinMaxScaler()
scaledData = scaler.fit_transform(df['open'].values.reshape(-1, 1))

def createTrainTestSplit(data, sequenceLength):
    X, Y = [], []
    for i in range(sequenceLength, len(data)):
        X.append(data[i-sequenceLength:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)


feature, target = createTrainTestSplit(scaledData, 50)

xTrain,xTest,yTrain,yTest = train_test_split(feature,
                                                target,
                                                test_size=0.2,
                                                random_state=42,
                                                shuffle=False)

xTrainReshaped = np.reshape(xTrain, (xTrain.shape[0], 1, xTrain.shape[1]))
xTestReshaped = np.reshape(xTest, (xTest.shape[0], 1, xTest.shape[1]))


In [6]:



def objective(trial):

    
    nLstmLayers = trial.suggest_int('n_lstm_layers', 1, 6)
    model = Sequential()
    for i in range(nLstmLayers):
        numHidden = trial.suggest_int(f'n_lstm_units_l{i}', 4, 512)
        if i == 0:
            model.add(LSTM(numHidden, return_sequences=True, input_shape=(xTrainReshaped.shape[1], xTrainReshaped.shape[2])))
        else:
            model.add(LSTM(numHidden,return_sequences=True))
    numLastLSTM = trial.suggest_int(f'n__lstm_units_last{i+1}',4,64)
    model.add(LSTM(numLastLSTM))

    nDenseLayers = trial.suggest_int(f'n_dense_layers',1,4)
    for i in range(nDenseLayers):
        numHidden = trial.suggest_int(f'n_units_d{i}', 4, 512)
        model.add(Dense(numHidden))
    
    model.add(Dense(1))
    learningRate = trial.suggest_float(
        "learning_rate_init", 1e-5, 1e-3, log=True
    )
    model.compile(optimizer=Adam(learning_rate=learningRate), loss='mean_squared_error')

    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min')
    batchSize = trial.suggest_categorical('batch_size', [4,16,64])
    history = model.fit(xTrainReshaped, yTrain, batch_size=batchSize, verbose=0,epochs=100, validation_data=(xTestReshaped, yTest), callbacks=[earlyStopping, reduceLr])

    valLoss = history.history['val_loss'][-1]

    return valLoss



In [7]:
storageUrl = "sqlite:///univariateStockStudy.db"
study = optuna.create_study(study_name="univariateAnalysis", storage=storageUrl, direction="minimize", load_if_exists=True)
study.optimize(objective, n_trials=10)

[I 2024-01-17 13:25:16,229] A new study created in RDB with name: univariateAnalysis



Epoch 8: ReduceLROnPlateau reducing learning rate to 1.5494966646656394e-05.
Restoring model weights from the end of the best epoch: 3.

Epoch 13: ReduceLROnPlateau reducing learning rate to 3.098993329331279e-06.
Epoch 13: early stopping


[I 2024-01-17 13:25:24,184] Trial 0 finished with value: 0.0038712986279278994 and parameters: {'n_lstm_layers': 4, 'n_lstm_units_l0': 162, 'n_lstm_units_l1': 136, 'n_lstm_units_l2': 212, 'n_lstm_units_l3': 33, 'n__lstm_units_last4': 55, 'n_dense_layers': 2, 'n_units_d0': 363, 'n_units_d1': 449, 'learning_rate_init': 7.747483401767999e-05, 'batch_size': 21}. Best is trial 0 with value: 0.0038712986279278994.



Epoch 17: ReduceLROnPlateau reducing learning rate to 0.00010900713969022036.

Epoch 27: ReduceLROnPlateau reducing learning rate to 2.180142764700577e-05.

Epoch 36: ReduceLROnPlateau reducing learning rate to 4.3602856749203054e-06.

Epoch 41: ReduceLROnPlateau reducing learning rate to 8.720571713638492e-07.
Restoring model weights from the end of the best epoch: 35.
Epoch 45: early stopping


[I 2024-01-17 13:26:16,012] Trial 1 finished with value: 0.0007606737199239433 and parameters: {'n_lstm_layers': 6, 'n_lstm_units_l0': 145, 'n_lstm_units_l1': 508, 'n_lstm_units_l2': 58, 'n_lstm_units_l3': 422, 'n_lstm_units_l4': 501, 'n_lstm_units_l5': 280, 'n__lstm_units_last6': 29, 'n_dense_layers': 1, 'n_units_d0': 256, 'learning_rate_init': 0.0005450357143680049, 'batch_size': 22}. Best is trial 1 with value: 0.0007606737199239433.


...

[I 2024-01-17 13:27:52,808] Trial 7 finished with value: 0.00485955411568284 and parameters: {'n_lstm_layers': 5, 'n_lstm_units_l0': 194, 'n_lstm_units_l1': 457, 'n_lstm_units_l2': 52, 'n_lstm_units_l3': 189, 'n_lstm_units_l4': 272, 'n__lstm_units_last5': 37, 'n_dense_layers': 4, 'n_units_d0': 7, 'n_units_d1': 191, 'n_units_d2': 77, 'n_units_d3': 136, 'learning_rate_init': 8.18379600008267e-05, 'batch_size': 36}. Best is trial 6 with value: 0.0006102996994741261.



Epoch 7: ReduceLROnPlateau reducing learning rate to 1.2021449219901116e-05.

Epoch 12: ReduceLROnPlateau reducing learning rate to 2.404289807600435e-06.

Epoch 17: ReduceLROnPlateau reducing learning rate to 4.80857943330193e-07.
Restoring model weights from the end of the best epoch: 8.
Epoch 18: early stopping


[I 2024-01-17 13:28:23,614] Trial 8 finished with value: 0.00550114456564188 and parameters: {'n_lstm_layers': 6, 'n_lstm_units_l0': 220, 'n_lstm_units_l1': 479, 'n_lstm_units_l2': 508, 'n_lstm_units_l3': 47, 'n_lstm_units_l4': 269, 'n_lstm_units_l5': 362, 'n__lstm_units_last6': 57, 'n_dense_layers': 1, 'n_units_d0': 480, 'learning_rate_init': 6.0107245253444946e-05, 'batch_size': 13}. Best is trial 6 with value: 0.0006102996994741261.



Epoch 18: ReduceLROnPlateau reducing learning rate to 2.9188555345172063e-06.

Epoch 23: ReduceLROnPlateau reducing learning rate to 5.837710887135473e-07.
Restoring model weights from the end of the best epoch: 14.
Epoch 24: early stopping


[I 2024-01-17 13:28:32,659] Trial 9 finished with value: 0.004087240435183048 and parameters: {'n_lstm_layers': 3, 'n_lstm_units_l0': 483, 'n_lstm_units_l1': 208, 'n_lstm_units_l2': 81, 'n__lstm_units_last3': 41, 'n_dense_layers': 2, 'n_units_d0': 191, 'n_units_d1': 365, 'learning_rate_init': 1.4594277242572434e-05, 'batch_size': 48}. Best is trial 6 with value: 0.0006102996994741261.


In [8]:

best_trial = study.best_trial

print("Best trial ID: {}".format(best_trial.number))
print("Value of the best objective (lowest validation loss): {}".format(best_trial.value))


print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))


Best trial ID: 6
Value of the best objective (lowest validation loss): 0.0006102996994741261
Best hyperparameters:
n_lstm_layers: 6
n_lstm_units_l0: 87
n_lstm_units_l1: 391
n_lstm_units_l2: 134
n_lstm_units_l3: 506
n_lstm_units_l4: 158
n_lstm_units_l5: 188
n__lstm_units_last6: 5
n_dense_layers: 3
n_units_d0: 264
n_units_d1: 259
n_units_d2: 275
learning_rate_init: 0.0005406486882754583
batch_size: 21


In [25]:
study.trials_dataframe().columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_batch_size', 'params_learning_rate_init',
       'params_n__lstm_units_last1', 'params_n__lstm_units_last2',
       'params_n__lstm_units_last3', 'params_n__lstm_units_last4',
       'params_n__lstm_units_last5', 'params_n__lstm_units_last6',
       'params_n_dense_layers', 'params_n_lstm_layers',
       'params_n_lstm_units_l0', 'params_n_lstm_units_l1',
       'params_n_lstm_units_l2', 'params_n_lstm_units_l3',
       'params_n_lstm_units_l4', 'params_n_lstm_units_l5', 'params_n_units_d0',
       'params_n_units_d1', 'params_n_units_d2', 'params_n_units_d3', 'state'],
      dtype='object')

In [28]:
def buildModel(trial):
    
    nLstmLayers = trial.suggest_int('n_lstm_layers', 1, 6)
    model = Sequential()
    for i in range(nLstmLayers):
        numHidden = trial.suggest_int(f'n_lstm_units_l{i}', 4, 512)
        if i == 0:
            model.add(LSTM(numHidden, return_sequences=True, input_shape=(xTrainReshaped.shape[1], xTrainReshaped.shape[2])))
        else:
            model.add(LSTM(numHidden,return_sequences=True))
    numLastLSTM = trial.suggest_int(f'n__lstm_units_last{i+1}',4,64)
    model.add(LSTM(numLastLSTM))

    nDenseLayers = trial.suggest_int(f'n_dense_layers',1,4)
    for i in range(nDenseLayers):
        numHidden = trial.suggest_int(f'n_units_d{i}', 4, 512)
        model.add(Dense(numHidden))
    
    model.add(Dense(1))
    learningRate = trial.suggest_float(
        "learning_rate_init", 1e-5, 1e-3, log=True
    )
    model.compile(optimizer=Adam(learning_rate=learningRate), loss='mean_squared_error')
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min')
    batchSize = trial.suggest_categorical('batch_size', [4,16,64])
    history = model.fit(xTrainReshaped, yTrain, batch_size=batchSize, verbose=1,epochs=100, validation_data=(xTestReshaped, yTest), callbacks=[earlyStopping, reduceLr])

    return model,history

In [19]:
model,history = buildModel(best_trial)

Epoch 1/100
75/75 [==============================] - 6s 24ms/step - loss: 0.0733 - val_loss: 0.2426 - lr: 5.4065e-04
Epoch 2/100
75/75 [==============================] - 1s 10ms/step - loss: 0.0454 - val_loss: 0.0145 - lr: 5.4065e-04
Epoch 3/100
75/75 [==============================] - 1s 11ms/step - loss: 0.0018 - val_loss: 0.0059 - lr: 5.4065e-04
Epoch 4/100
75/75 [==============================] - 1s 11ms/step - loss: 0.0017 - val_loss: 0.0118 - lr: 5.4065e-04
Epoch 5/100
75/75 [==============================] - 1s 16ms/step - loss: 0.0012 - val_loss: 0.0038 - lr: 5.4065e-04
Epoch 6/100
75/75 [==============================] - 1s 12ms/step - loss: 9.1785e-04 - val_loss: 0.0028 - lr: 5.4065e-04
Epoch 7/100
75/75 [==============================] - 1s 13ms/step - loss: 6.5936e-04 - val_loss: 0.0017 - lr: 5.4065e-04
Epoch 8/100
75/75 [==============================] - 1s 13ms/step - loss: 0.0012 - val_loss: 0.0033 - lr: 5.4065e-04
Epoch 9/100
75/75 [==============================] - 1s 

In [26]:
predictions = model.predict(xTestReshaped)


dates = df['date'].values[-len(yTest):]  


data = {
    "date": dates,
    "predictedOpen": predictions.flatten(),
    "open": yTest
}

dfPredicted = pd.DataFrame(data)
dfPredicted["date"] = pd.to_datetime(dfPredicted["date"])
dfPredicted['predictedOpen'] = scaler.inverse_transform(dfPredicted['predictedOpen'].values.reshape(-1, 1))
dfPredicted['open'] = scaler.inverse_transform(dfPredicted['open'].values.reshape(-1, 1))


MAE = mean_absolute_error(dfPredicted["predictedOpen"], dfPredicted["open"])
print(f'Mean Absolute Error (MAE): {np.round(MAE, 2)}')

MSE = mean_squared_error( dfPredicted["open"],dfPredicted["predictedOpen"])
print(f'Mean Squared Error (MAE): {np.round(MSE, 2)}')

MAPE = np.mean(np.abs((dfPredicted["open"] - dfPredicted["predictedOpen"]) / dfPredicted["open"])) * 100
print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)} %')


MDAPE = np.median(np.abs((dfPredicted["open"] - dfPredicted["predictedOpen"]) / dfPredicted["open"])) * 100
print(f'Median Absolute Percentage Error (MDAPE): {np.round(MDAPE, 2)} %')


13/13 [==============================] - 0s 5ms/step
Mean Absolute Error (MAE): 4.53
Mean Squared Error (MAE): 29.52
Mean Absolute Percentage Error (MAPE): 2.76 %
Median Absolute Percentage Error (MDAPE): 2.68 %


In [46]:
historyDf = pd.DataFrame(history.history).reset_index()
historyDf.rename(columns={'index':'epoch'},inplace=True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=historyDf["epoch"], y=historyDf["loss"],name='Train Loss',line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=historyDf["epoch"], y=historyDf["val_loss"],name='Validation Loss',line=dict(color='red', width=1)))
fig.update_layout(height=400, width=1200, title_text=f"Learning Curve", showlegend=True)

In [47]:
fig = make_subplots()


fig.add_trace(go.Scatter(x=dfPredicted['date'], y=dfPredicted['open'], 
                         mode='lines', name='Actual'))


fig.add_trace(go.Scatter(x=dfPredicted['date'], y=dfPredicted['predictedOpen'], 
                         mode='lines', name='Predicted'))


fig.add_trace(go.Scatter(x=dfPredicted['date'].tolist() + dfPredicted['date'].tolist()[::-1],
                         y=dfPredicted['open'].tolist() + dfPredicted['predictedOpen'].tolist()[::-1],
                         fill='toself',
                         fillcolor='grey',
                         opacity=0.3,
                         line=dict(color='rgba(255,255,255,0)'),
                         showlegend=False))


fig.update_layout(title='Predicted vs Actual Over Test Data',
                  xaxis_title='Date',
                  yaxis_title='Value',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor="white")


fig.update_xaxes(tickangle=45)


fig.show()